In [24]:
BODY_PARTS_KPT_IDS = [[1, 2], [1, 5], [2, 3], [3, 4], [5, 6], [6, 7], [1, 8], [8, 9], [9, 10], [1, 11],
                      [11, 12], [12, 13], [1, 0], [0, 14], [14, 16], [0, 15], [15, 17], [2, 16], [5, 17]]
BODY_PARTS_PAF_IDS = ([12, 13], [20, 21], [14, 15], [16, 17], [22, 23], [24, 25], [0, 1], [2, 3], [4, 5],
                      [6, 7], [8, 9], [10, 11], [28, 29], [30, 31], [34, 35], [32, 33], [36, 37], [18, 19], [26, 27])


def display_image(pose_entries, all_keypoints, scale, pad, img, orig_img):
    stride = 8
    upsample_ratio = 4
    color = [0, 224, 255]

    for kpt_id in range(all_keypoints.shape[0]):
        all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
        all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
    for n in range(len(pose_entries)):
        if len(pose_entries[n]) == 0:
            continue
        for part_id in range(len(BODY_PARTS_PAF_IDS) - 2):
            kpt_a_id = BODY_PARTS_KPT_IDS[part_id][0]
            global_kpt_a_id = pose_entries[n][kpt_a_id]
            if global_kpt_a_id != -1:
                x_a, y_a = all_keypoints[int(global_kpt_a_id), 0:2]
                cv2.circle(img, (int(x_a), int(y_a)), 3, color, -1)
            kpt_b_id = BODY_PARTS_KPT_IDS[part_id][1]
            global_kpt_b_id = pose_entries[n][kpt_b_id]
            if global_kpt_b_id != -1:
                x_b, y_b = all_keypoints[int(global_kpt_b_id), 0:2]
                cv2.circle(img, (int(x_b), int(y_b)), 3, color, -1)
            if global_kpt_a_id != -1 and global_kpt_b_id != -1:
                cv2.line(img, (int(x_a), int(y_a)), (int(x_b), int(y_b)), color, 2)

    img = cv2.addWeighted(orig_img, 0.6, img, 0.4, 0)
    #cv2.imshow('Lightweight Human Pose Estimation Python Demo', img)
    cv2.imwrite('out2.jpg',img)

In [ ]:
from datetime import datetime as dt
import requests
import json
import cv2
import numpy as np

stamp = dt.now().strftime("%Y%m%d%H%M%S")
#image_file = 'eagles.jpg'
image_file = 'bdawk.jpg'
url = 'http://ec2-3-87-216-77.compute-1.amazonaws.com:8080/invocations'
#image_display = skimage.io.imread(image_file)

#payload = open(image_file)
with open(image_file, 'rb') as f:
  payload = f.read()


headers = {'content-type': 'image/jpeg'}

#get the inferences
r = requests.post(url, data=payload, headers=headers)
results =  json.loads(r.text)

#print (json.dumps(results))

image = cv2.imread(image_file, cv2.IMREAD_COLOR)
orig_img = image.copy()


pose_entries = results['prediction']['pose_entries']
all_keypoints = results['prediction']['all_keypoints']
pad = results['prediction']['pad']
scale = results['prediction']['scale']


display_image(pose_entries,np.array(all_keypoints),scale,pad,image,orig_img)
